## Polynomial Chaos Expansion example: Friedman function (5 random inputs, scalar output)

Author: Katiana Kontolati, Dimitrios Loukrezis \
Date: May 7, 2021

In this example, PCE is used to generate a surrogate model for a given set of 5D data.

### Friedman function

$$ f(x) = 10 sin(\pi x_1x_2) + 20(x_3 - 0.5)^2 + 10x_4 + 5x_5$$

**Description:**  Dimensions: 5

**Input Domain:**  This function is evaluated on the hypercube $x_i \in [0,1]$ for all i = 1, …, 5.

**Reference:**  Friedman, J. H. (1991). Multivariate adaptive regression splines. The annals of statistics, 19(1), 1-67.

Import necessary libraries.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from UQpy.Distributions import Uniform, JointInd
from UQpy.Surrogates import *

Define the function.

In [2]:
def function(x):
    return 10*np.sin(np.pi*x[:,0]*x[:,1]) + 20*(x[:,2]-0.5)**2 + 10*x[:,3] + 5*x[:,4]

Create a distribution object, generate samples and evaluate the function at the samples.

In [7]:
np.random.seed(1)

dist = Uniform(loc=0, scale=1)  

marg = [dist]*5
joint = JointInd(marginals=marg)

n_samples = 200
x = joint.rvs(n_samples)
y = function(x)  

Create an object from the PCE class.

In [8]:
max_degree = 3
pce = PolyChaosExp(joint)
construct_td_basis(pce, max_degree)

Compute PCE coefficients using least squares regression.

In [9]:
fit_lstsq(pce, x, y)

Compute PCE coefficients using Lasso regression.

In [10]:
pce2 = PolyChaosExp(joint)
construct_td_basis(pce2, max_degree)
fit_lasso(pce2, x, y)

Compute PCE coefficients using Ridge regression.

In [11]:
pce3 = PolyChaosExp(joint)
construct_td_basis(pce3, max_degree)
fit_ridge(pce3, x, y)

### Error Estimation

Validation error.

In [13]:
n_samples = 100
x_val = joint.rvs(n_samples)
y_val = function(x_val)

y_pce = pce.predict(x_val).flatten()
y_pce2 = pce2.predict(x_val).flatten()
y_pce3 = pce3.predict(x_val).flatten()

error = np.sum(np.abs(y_pce - y_val)/np.abs(y_val))/n_samples
error2 = np.sum(np.abs(y_pce2 - y_val)/np.abs(y_val))/n_samples
error3 = np.sum(np.abs(y_pce3 - y_val)/np.abs(y_val))/n_samples

print('Validation error, LSTSQ-PCE:', error)
print('Validation error, LASSO-PCE:', error2)
print('Validation error, Ridge-PCE:', error3)

Validation error, LSTSQ-PCE: 0.020401979267738377
Validation error, LASSO-PCE: 0.019127963648936673
Validation error, Ridge-PCE: 0.019832354802127902


### Moment Estimation

Returns mean and variance of the PCE surrogate.

In [14]:
n_mc = 1000000
x_mc = joint.rvs(n_mc)
y_mc = function(x_mc)    
mean_mc = np.mean(y_mc)
var_mc = np.var(y_mc)

print('Moments from least squares regression :', pce_mean(pce), pce_variance(pce))
print('Moments from LASSO regression :', pce_mean(pce2), pce_variance(pce2))
print('Moments from Ridge regression :', pce_mean(pce3), pce_variance(pce3))
print('Moments from MC integration: ', mean_mc, var_mc)

Moments from least squares regression : [14.38627129] [23.80629216]
Moments from LASSO regression : [14.38477638] [23.63639382]
Moments from Ridge regression : [14.36557952] [23.38819865]
Moments from MC integration:  14.413797025098804 23.851668165190734
